In [99]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
import pandas as pd
import numpy as np


import sys
sys.path.insert(0, "../scripts")
import ml_window as m 
import helpers as h

import json 


In [100]:
# macro f1 score...
with open("../constants/svm_metrics_230704.json") as f:
    svm_data =json.load(f)

with open("../constants/ewm_metrics_230704.json") as f:
    ewm_data =json.load(f)

In [101]:
svm_data["exp_A"]["0"]

{'name': ['amb_minus_meas_temp',
  'meas_minus_deriv_temp',
  'meas_minus_deriv_rh'],
 'metrics': {'standard': {'macro avg f1-score': 0.574},
  'drdr': {'unbounded acc': -10.88, 'bounded acc': 0.1},
  'custom': {'(hits + near hits)/guesses': 0.595, 'guesses/actions': 2.846}}}

In [102]:
def sort_and_return_quick_result(all_results, data_type="svm"):
    print(data_type)
    if data_type=="svm":
        sorted_results = sorted(all_results.items(), key=lambda x:x[1]["metrics"]["standard"]["macro avg f1-score"], reverse=True)
        quick_res = [
            {
            "name": v["name"],
            "val": v["metrics"]["standard"]["macro avg f1-score"] 
            }
            for k,v in sorted_results]
    else:
        sorted_results = sorted(all_results.items(), key=lambda x:x[1]["standard"]["macro avg"]["f1-score"], reverse=True)
        quick_res = [
        {
            "name": k,
            "val":  v["standard"]["macro avg"]["f1-score"],
        }
        for k,v in sorted_results]
    return quick_res, sorted_results 

In [163]:
quick, slow = sort_and_return_quick_result(ewm_data["exp_A"], "ewm")
quick

ewm


[{'name': 'z=1', 'val': 0.7242424242424242},
 {'name': 'z=2', 'val': 0.5824526404952461}]

In [ ]:
[i for i in ]

In [104]:
svm_data["exp_A"]["0"]["metrics"]["standard"]["macro avg f1-score"]

0.574

In [110]:
quick, slow = sort_and_return_quick_result(svm_data["exp_A"], "svm")
# quick

svm


In [106]:
exps = list(svm_data.keys())
exps

['exp_A', 'exp_B', 'exp_C']

In [152]:
data_pts["exp_A"]

[{'name': 'z=1', 'val': 0.7242424242424242, 'chart_name': 'EWM'},
 {'name': ['dt_meas_temp', 'dt_meas_rh'], 'val': 0.709, 'chart_name': 'SVM_1'},
 {'name': ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_amb_rh'],
  'val': 0.699,
  'chart_name': 'SVM_2'},
 {'name': ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_amb_temp'],
  'val': 0.694,
  'chart_name': 'SVM_3'}]

In [166]:
res = filter(lambda x: True if x["name"] == "z=1" else False, data_pts["exp_A"])
print(list(res))

[{'name': 'z=1', 'val': 0.7242424242424242, 'chart_name': 'EWM'}]


In [178]:
data_pts = {}
for exp in exps:
    data_pts[exp] = []
    for type, data in zip(["ewm", "svm"], [ewm_data, svm_data]):
        quick, slow = sort_and_return_quick_result(data[exp], type)
        if type == "ewm":
            res = list(filter(lambda x: True if x["name"] == "z=1" else False, quick))
            data_pts[exp].append(res[0])
            data_pts[exp][0]["chart_name"] = "EWM"
        else:
            data_pts[exp].extend(quick[0:3])
            for i in range(3):
                data_pts[exp][i+1]["chart_name"] = f"SVM_{i+1}"








ewm
svm
ewm
svm
ewm
svm


In [179]:
data_pts["exp_A"]

[{'name': 'z=1', 'val': 0.7242424242424242, 'chart_name': 'EWM'},
 {'name': ['dt_meas_temp', 'dt_meas_rh'], 'val': 0.709, 'chart_name': 'SVM_1'},
 {'name': ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_amb_rh'],
  'val': 0.699,
  'chart_name': 'SVM_2'},
 {'name': ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_amb_temp'],
  'val': 0.694,
  'chart_name': 'SVM_3'}]

In [180]:
for k,v in data_pts.items():
    for small_dict in v:
        small_dict["exp"] = k

In [181]:
data_list = sum(data_pts.values(), [])

In [182]:
df = pd.DataFrame(data_list)
df

,name,val,chart_name,exp
0,z=1,0.724242,EWM,exp_A
1,"[dt_meas_temp, dt_meas_rh]",0.709000,SVM_1,exp_A
2,"[meas_temp, meas_rh, dt_meas_temp, dt_amb_rh]",0.699000,SVM_2,exp_A
3,"[meas_temp, meas_rh, dt_meas_temp, dt_amb_temp]",0.694000,SVM_3,exp_A
4,z=1,0.461721,EWM,exp_B
5,[amb_rh],0.445000,SVM_1,exp_B
6,[amb_temp],0.424000,SVM_2,exp_B
7,"[amb_temp, dt_meas_temp, dt_amb_temp, dt_amb_rh]",0.419000,SVM_3,exp_B
8,z=1,0.437793,EWM,exp_C
9,"[amb_temp, amb_rh]",0.626000,SVM_1,exp_C


In [190]:
fig = px.strip(df, x="chart_name", y="val", color="exp")
fig.update_layout(title="Macro Average F1 Score")
fig.show()

In [185]:
df.groupby("exp")["val"]

In [108]:
quick, slow = sort_and_return_quick_result(svm_data["exp_A"], "svm")
quick

svm


[{'name': ['dt_meas_temp', 'dt_meas_rh'], 'val': 0.709},
 {'name': ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_amb_rh'], 'val': 0.699},
 {'name': ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_amb_temp'],
  'val': 0.694},
 {'name': ['meas_temp', 'dt_meas_temp', 'dt_meas_rh', 'dt_amb_rh'],
  'val': 0.686},
 {'name': ['meas_temp', 'dt_meas_temp', 'dt_meas_rh', 'dt_amb_temp'],
  'val': 0.686},
 {'name': ['meas_temp', 'dt_meas_temp', 'dt_amb_temp', 'dt_amb_rh'],
  'val': 0.684},
 {'name': ['meas_temp', 'amb_rh', 'dt_meas_temp', 'dt_meas_rh'], 'val': 0.673},
 {'name': ['dt_meas_temp', 'dt_meas_rh', 'dt_amb_temp'], 'val': 0.668},
 {'name': ['meas_temp', 'amb_rh', 'dt_meas_temp', 'dt_amb_rh'], 'val': 0.668},
 {'name': ['meas_temp', 'meas_rh', 'dt_meas_temp', 'dt_meas_rh'],
  'val': 0.662},
 {'name': ['dt_meas_temp', 'dt_meas_rh', 'dt_amb_rh'], 'val': 0.662},
 {'name': ['amb_rh', 'dt_meas_temp', 'dt_meas_rh', 'dt_amb_temp'],
  'val': 0.66},
 {'name': ['dt_meas_rh'], 'val': 0.66},
 {'name': ['me

In [109]:
svm_data["exp_A"]

{'0': {'name': ['amb_minus_meas_temp',
   'meas_minus_deriv_temp',
   'meas_minus_deriv_rh'],
  'metrics': {'standard': {'macro avg f1-score': 0.574},
   'drdr': {'unbounded acc': -10.88, 'bounded acc': 0.1},
   'custom': {'(hits + near hits)/guesses': 0.595, 'guesses/actions': 2.846}}},
 '1': {'name': ['meas_temp', 'meas_rh', 'amb_temp', 'dt_meas_rh'],
  'metrics': {'standard': {'macro avg f1-score': 0.534},
   'drdr': {'unbounded acc': -15.23, 'bounded acc': 0.0},
   'custom': {'(hits + near hits)/guesses': 0.628, 'guesses/actions': 3.308}}},
 '2': {'name': ['dt_meas_rh', 'dt_amb_temp'],
  'metrics': {'standard': {'macro avg f1-score': 0.618},
   'drdr': {'unbounded acc': -16.11, 'bounded acc': 0.0},
   'custom': {'(hits + near hits)/guesses': 0.475, 'guesses/actions': 9.231}}},
 '3': {'name': ['meas_rh', 'amb_temp', 'dt_meas_temp', 'dt_amb_temp'],
  'metrics': {'standard': {'macro avg f1-score': 0.581},
   'drdr': {'unbounded acc': -15.26, 'bounded acc': 0.01889},
   'custom': {'(hi